In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [19]:
import json
import math
import os
from pathlib import Path
from typing import List
import sys

import dataclass_array as dca
import jax.numpy as jnp
import matplotlib.pyplot as plt
import mitsuba as mi
import numpy as np
from PIL import Image
import pycolmap
import pyrender
import trimesh
import visu3d as v3d

sys.path.append(os.path.abspath(os.path.join("..", "bop_toolkit")))
from bop_toolkit.bop_toolkit_lib.pose_error import vsd, mssd, mspd, add
from bop_toolkit.bop_toolkit_lib.misc import get_symmetry_transformations
from bop_toolkit.bop_toolkit_lib.renderer import create_renderer

import burybarrel.colmap_util as cutil
from burybarrel.image import render_v3d

In [ ]:
renderer = create_renderer(1920, 875, renderer_type="vispy", mode="depth")

In [25]:
objpath = Path("/scratch/jeyan/barreldata/models3d/barrelsingle-scaled.ply")
objinfopath = Path("/scratch/jeyan/barreldata/models3d/model_info.json")
caminfopath = Path("/scratch/jeyan/barreldata/divedata/dive8/barrelddt1/camera.json")
gtposepath = Path("/scratch/jeyan/barreldata/divedata/dive8/barrelddt1/gt_obj2cam.json")
masksdir = Path("/scratch/jeyan/barreldata/divedata/dive8/barrelddt1/mask")
estimatedpaths = {
    "barrelddt1_renders_layer9": Path("/scratch/jeyan/foundpose/output_barrelddt1_renders_layer9/inference/estimated-poses.json"),
}

with open(gtposepath, "rt") as f:
    gtposes = json.load(f)
with open(objinfopath, "rt") as f:
    objinfo = json.load(f)
with open(caminfopath, "rt") as f:
    caminfo = json.load(f)
maskpaths = list(sorted(masksdir.glob("*.png")))

renderer = create_renderer(1920, 875, renderer_type="vispy", mode="depth")
renderer.add_object("barrel", objpath)

In [54]:
gtposes = list(sorted(gtposes, key=lambda x: int(x["img_id"])))
gt_Rs = np.array([gtpose["R"] for gtpose in gtposes])
gt_ts = np.array([gtpose["t"] for gtpose in gtposes])[..., None]
imgids = [gtpose["img_id"] for gtpose in gtposes]

In [55]:
K = np.array([
    [caminfo["fx"], 0, caminfo["cx"]],
    [0, caminfo["fy"], caminfo["cy"]],
    [0, 0, 1.0]
])

In [56]:
symTs = get_symmetry_transformations(objinfo["barrelsingle-scaled.ply"], 0.01)

In [57]:
mesh = trimesh.load(objpath)
vtxs = np.array(mesh.vertices)

In [58]:
fx, fy, cx, cy = K[0, 0], K[1, 1], K[0, 2], K[1, 2]
R_gt = gt_Rs[0]
t_gt = gt_ts[0]
depth_test = renderer.render_object("barrel", R_gt, t_gt, fx, fy, cx, cy)["depth"]

In [74]:
estpath = estimatedpaths["barrelddt1_renders_layer9"]
with open(estpath, "rt") as f:
    ests = json.load(f)
allvsd = []
allmssd = []
allmspd = []
for i, imgid in enumerate(imgids):
    R_gt = gt_Rs[i]
    t_gt = gt_ts[i]
    # no test depth, just use ground truth
    depth_test = renderer.render_object("barrel", R_gt, t_gt, fx, fy, cx, cy)["depth"]
    imgmatches = list(filter(lambda x: x["img_id"] == imgid, ests))
    R_est = np.array(imgmatches[0]["R"])
    t_est = np.array(imgmatches[0]["t"])
    vsdres = vsd(R_est, t_est, R_gt, t_gt, depth_test, K, 0.2, [0.1, 0.2, 0.3, 0.4, 0.5], False, None, renderer, "barrel", "step")
    mssdres = mssd(R_est, t_est, R_gt, t_gt, vtxs, symTs)
    mspdres = mspd(R_est, t_est, R_gt, t_gt, K, vtxs, symTs)
    allvsd.append(vsdres)
    allmssd.append(mssdres)
    allmspd.append(mspdres)

In [ ]:
i = 7
allvsd[i], allmssd[i], allmspd[i]

In [ ]:
vsdres